# Experiment 4: Criterion
- Cross Entrophy Loss
- Label Smoothing Loss
- F1 Loss
- Focal Loss

In [1]:
import os
import sys
import pickle
from glob import glob
from time import time
from tqdm.notebook import tqdm
from collections import Counter

# scikit-learn
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

# data processing
import cv2
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

# pytorch
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models, utils
torch.manual_seed(0)
print(f'Pytorch version: {torch.__version__}')

# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'This notebook use {device}')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Pytorch version: 1.6.0
This notebook use cuda:0


In [2]:
# 파일 경로 사용자 정의
class path:
    data = '../input/data'
    train = '../input/data/train'
    train_img = f'{train}/images'
    train_df = f'{train}/train.csv'
    test = '../input/data/eval'
    test_img = f'{test}/images'
    test_df = f'{test}/info.csv'

In [3]:
BATCH_SIZE=16
NUM_WORKERS=2
LEARNING_RATE=1e-4
EPOCHS=3

## 1. Dataset

In [4]:
class MaskDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def set_transform(self, transform):
        self.transform = transform
    
    def __getitem__(self, index):
        data = self.df.iloc[index]
        target = data.target
        image = Image.open(data.path)
        
        if self.transform:
            image = self.transform(image)
        
        return image, target
        
    def __len__(self):
        return len(self.df)

In [5]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [6]:
train_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.,)
])

In [7]:
valid_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

## 2. Modeling

In [8]:
import math
from typing import TYPE_CHECKING, Any, Callable, Optional

import torch
import torch.optim

if TYPE_CHECKING:
    from torch.optim.optimizer import _params_t
else:
    _params_t = Any


class MADGRAD(torch.optim.Optimizer):
    """
    MADGRAD_: A Momentumized, Adaptive, Dual Averaged Gradient Method for Stochastic 
    Optimization.
    .. _MADGRAD: https://arxiv.org/abs/2101.11075
    MADGRAD is a general purpose optimizer that can be used in place of SGD or
    Adam may converge faster and generalize better. Currently GPU-only.
    Typically, the same learning rate schedule that is used for SGD or Adam may
    be used. The overall learning rate is not comparable to either method and
    should be determined by a hyper-parameter sweep.
    MADGRAD requires less weight decay than other methods, often as little as
    zero. Momentum values used for SGD or Adam's beta1 should work here also.
    On sparse problems both weight_decay and momentum should be set to 0.
    Arguments:
        params (iterable): 
            Iterable of parameters to optimize or dicts defining parameter groups.
        lr (float): 
            Learning rate (default: 1e-2).
        momentum (float): 
            Momentum value in  the range [0,1) (default: 0.9).
        weight_decay (float): 
            Weight decay, i.e. a L2 penalty (default: 0).
        eps (float): 
            Term added to the denominator outside of the root operation to improve numerical stability. (default: 1e-6).
    """

    def __init__(
        self, params: _params_t, lr: float = 1e-2, momentum: float = 0.9, weight_decay: float = 0, eps: float = 1e-6,
    ):
        if momentum < 0 or momentum >= 1:
            raise ValueError(f"Momentum {momentum} must be in the range [0,1]")
        if lr <= 0:
            raise ValueError(f"Learning rate {lr} must be positive")
        if weight_decay < 0:
            raise ValueError(f"Weight decay {weight_decay} must be non-negative")
        if eps < 0:
            raise ValueError(f"Eps must be non-negative")

        defaults = dict(lr=lr, eps=eps, momentum=momentum, weight_decay=weight_decay)
        super().__init__(params, defaults)

    @property
    def supports_memory_efficient_fp16(self) -> bool:
        return False

    @property
    def supports_flat_params(self) -> bool:
        return True

    def step(self, closure: Optional[Callable[[], float]] = None) -> Optional[float]:
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        # step counter must be stored in state to ensure correct behavior under
        # optimizer sharding
        if 'k' not in self.state:
            self.state['k'] = torch.tensor([0], dtype=torch.long)
        k = self.state['k'].item()

        for group in self.param_groups:
            eps = group["eps"]
            lr = group["lr"] + eps
            decay = group["weight_decay"]
            momentum = group["momentum"]

            ck = 1 - momentum
            lamb = lr * math.pow(k + 1, 0.5)

            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                if "grad_sum_sq" not in state:
                    state["grad_sum_sq"] = torch.zeros_like(p.data).detach()
                    state["s"] = torch.zeros_like(p.data).detach()
                    if momentum != 0:
                        state["x0"] = torch.clone(p.data).detach()

                if momentum != 0.0 and grad.is_sparse:
                    raise RuntimeError("momentum != 0 is not compatible with sparse gradients")

                grad_sum_sq = state["grad_sum_sq"]
                s = state["s"]

                # Apply weight decay
                if decay != 0:
                    if grad.is_sparse:
                        raise RuntimeError("weight_decay option is not compatible with sparse gradients")

                    grad.add_(p.data, alpha=decay)

                if grad.is_sparse:
                    grad = grad.coalesce()
                    grad_val = grad._values()

                    p_masked = p.sparse_mask(grad)
                    grad_sum_sq_masked = grad_sum_sq.sparse_mask(grad)
                    s_masked = s.sparse_mask(grad)

                    # Compute x_0 from other known quantities
                    rms_masked_vals = grad_sum_sq_masked._values().pow(1 / 3).add_(eps)
                    x0_masked_vals = p_masked._values().addcdiv(s_masked._values(), rms_masked_vals, value=1)

                    # Dense + sparse op
                    grad_sq = grad * grad
                    grad_sum_sq.add_(grad_sq, alpha=lamb)
                    grad_sum_sq_masked.add_(grad_sq, alpha=lamb)

                    rms_masked_vals = grad_sum_sq_masked._values().pow_(1 / 3).add_(eps)

                    s.add_(grad, alpha=lamb)
                    s_masked._values().add_(grad_val, alpha=lamb)

                    # update masked copy of p
                    p_kp1_masked_vals = x0_masked_vals.addcdiv(s_masked._values(), rms_masked_vals, value=-1)
                    # Copy updated masked p to dense p using an add operation
                    p_masked._values().add_(p_kp1_masked_vals, alpha=-1)
                    p.data.add_(p_masked, alpha=-1)
                else:
                    if momentum == 0:
                        # Compute x_0 from other known quantities
                        rms = grad_sum_sq.pow(1 / 3).add_(eps)
                        x0 = p.data.addcdiv(s, rms, value=1)
                    else:
                        x0 = state["x0"]

                    # Accumulate second moments
                    grad_sum_sq.addcmul_(grad, grad, value=lamb)
                    rms = grad_sum_sq.pow(1 / 3).add_(eps)

                    # Update s
                    s.data.add_(grad, alpha=lamb)

                    # Step
                    if momentum == 0:
                        p.data.copy_(x0.addcdiv(s, rms, value=-1))
                    else:
                        z = x0.addcdiv(s, rms, value=-1)

                        # p is a moving average of z
                        p.data.mul_(1 - ck).add_(z, alpha=ck)


        self.state['k'] += 1
        return loss

### 2.1. Cross Entropy Loss

In [9]:
model1 = models.resnet18(pretrained=False)
n_features = model1.fc.in_features
model1.fc = nn.Linear(n_features, 18)
model1 = model1.cuda()

optimizer1 = MADGRAD(model1.parameters(), lr=LEARNING_RATE)
criterion1 = nn.CrossEntropyLoss().to(device)

### 2.2. Label Smoothing Loss

In [10]:
# src: https://github.com/pytorch/pytorch/issues/7455
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [11]:
model2 = models.resnet18(pretrained=False)
n_features = model2.fc.in_features
model2.fc = nn.Linear(n_features, 18)
model2 = model2.cuda()

optimizer2 = MADGRAD(model2.parameters(), lr=LEARNING_RATE)
criterion2 = LabelSmoothingLoss(classes=18, smoothing=0.1).to(device)

### 2.3. Focal Loss

In [12]:
# src: https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/8
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [13]:
model3 = models.resnet18(pretrained=False)
n_features = model3.fc.in_features
model3.fc = nn.Linear(n_features, 18)
model3 = model3.cuda()

optimizer3 = MADGRAD(model3.parameters(), lr=LEARNING_RATE)
criterion3 = FocalLoss().to(device)

### 2.4. F1 Loss

In [24]:
# src: https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1_Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

In [25]:
model4 = models.resnet18(pretrained=False)
n_features = model4.fc.in_features
model4.fc = nn.Linear(n_features, 18)
model4 = model4.cuda()

optimizer4 = MADGRAD(model4.parameters(), lr=LEARNING_RATE)
criterion4 = F1_Loss(classes=18).to(device)

## 3. Training

In [18]:
def test_eval(model, valid_dataset):
    model.eval()
    with torch.no_grad():
        y_true, y_pred = [], []
        for image, label in tqdm(valid_dataset):
            X = image.view(-1, 3, 224, 224).float().to(device)
            y = label.item()
            _, pred = torch.max(model(X), 1)
            pred = pred.item()
            y_true.append(y)
            y_pred.append(pred)
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
    model.train()
    return f1, accuracy

In [19]:
def train_model(train, test, model, criterion, optimizer, print_every=1):
    best_accuracy = 0
    for epoch in range(EPOCHS):
        loss_sum = 0
        for images, label in tqdm(train):
            X = images.view(-1, 3, 224, 224).float().to(device)
            y = label.to(device)
            
            y_pred = model(X)
            loss = criterion(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
            loss_sum += loss
          
        if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
            loss_avg = loss_sum / len(train)
            f1, accuracy = test_eval(model, test)
            print(f">> epoch:[{epoch+1}/{EPOCHS}] cost:{loss_avg:5.3f} test_accuracy:{accuracy:5.3f} test_f1_score:{f1:5.3f}") 

In [20]:
def cross_validation(df, model, criterion, optimizer, k_folds=5):
    skf = StratifiedKFold(n_splits=5)
    n_iter = 0
    for train_idx, valid_idx in skf.split(df, df.target):
        n_iter += 1
        print(f'>> Cross Validation {n_iter} Starts!')
        train, valid = df.loc[train_idx], df.loc[valid_idx]
        train_dataset, valid_dataset = MaskDataset(train), MaskDataset(valid)
        
        # augmentation 설정
        train_dataset.set_transform(train_transforms)
        valid_dataset.set_transform(valid_transforms)

        # 데이터로더 생성
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
        valid_loader = DataLoader(valid_dataset, shuffle=False)
        
        train_model(train_loader, valid_loader, model, criterion, optimizer)
        print()

In [21]:
df = pd.read_csv(f'{path.train}/train_modified.csv')[['path', 'target']]

### 3.1. Cross Entropy Loss

In [29]:
cross_validation(df, model1, criterion1, optimizer1)

>> Cross Validation 1 Starts!



>> epoch:[1/3] cost:1.558 test_accuracy:0.616 test_f1_score:0.388



>> epoch:[2/3] cost:0.873 test_accuracy:0.710 test_f1_score:0.539



>> epoch:[3/3] cost:0.691 test_accuracy:0.712 test_f1_score:0.571

>> Cross Validation 2 Starts!



>> epoch:[1/3] cost:0.602 test_accuracy:0.664 test_f1_score:0.618


...


>> epoch:[2/3] cost:0.249 test_accuracy:0.882 test_f1_score:0.790



>> epoch:[3/3] cost:0.221 test_accuracy:0.833 test_f1_score:0.751

>> Cross Validation 5 Starts!



>> epoch:[1/3] cost:0.238 test_accuracy:0.950 test_f1_score:0.909



>> epoch:[2/3] cost:0.186 test_accuracy:0.892 test_f1_score:0.821



>> epoch:[3/3] cost:0.153 test_accuracy:0.915 test_f1_score:0.851



### 3.2. Label Smoothing Loss

In [30]:
cross_validation(df, model2, criterion2, optimizer2)

>> Cross Validation 1 Starts!



>> epoch:[1/3] cost:1.864 test_accuracy:0.534 test_f1_score:0.312



>> epoch:[2/3] cost:1.357 test_accuracy:0.714 test_f1_score:0.505



>> epoch:[3/3] cost:1.187 test_accuracy:0.760 test_f1_score:0.559

>> Cross Validation 2 Starts!



>> epoch:[1/3] cost:1.100 test_accuracy:0.786 test_f1_score:0.598


...


>> epoch:[2/3] cost:0.809 test_accuracy:0.920 test_f1_score:0.847



>> epoch:[3/3] cost:0.772 test_accuracy:0.899 test_f1_score:0.807

>> Cross Validation 5 Starts!



>> epoch:[1/3] cost:0.781 test_accuracy:0.950 test_f1_score:0.916



>> epoch:[2/3] cost:0.749 test_accuracy:0.930 test_f1_score:0.874



>> epoch:[3/3] cost:0.725 test_accuracy:0.915 test_f1_score:0.851



### 3.3. Focal Loss

In [22]:
cross_validation(df, model3, criterion3, optimizer3)

>> Cross Validation 1 Starts!



>> epoch:[1/3] cost:1.146 test_accuracy:0.538 test_f1_score:0.377



>> epoch:[2/3] cost:0.518 test_accuracy:0.698 test_f1_score:0.539



>> epoch:[3/3] cost:0.374 test_accuracy:0.605 test_f1_score:0.514

>> Cross Validation 2 Starts!



>> epoch:[1/3] cost:0.316 test_accuracy:0.794 test_f1_score:0.657


...


>> epoch:[2/3] cost:0.118 test_accuracy:0.891 test_f1_score:0.768



>> epoch:[3/3] cost:0.093 test_accuracy:0.874 test_f1_score:0.754

>> Cross Validation 5 Starts!



>> epoch:[1/3] cost:0.109 test_accuracy:0.908 test_f1_score:0.855



>> epoch:[2/3] cost:0.077 test_accuracy:0.911 test_f1_score:0.843



>> epoch:[3/3] cost:0.069 test_accuracy:0.921 test_f1_score:0.868



### 3.4. F1 Loss

In [26]:
cross_validation(df, model4, criterion4, optimizer4)

>> Cross Validation 1 Starts!



>> epoch:[1/3] cost:0.895 test_accuracy:0.389 test_f1_score:0.195



>> epoch:[2/3] cost:0.792 test_accuracy:0.505 test_f1_score:0.321



>> epoch:[3/3] cost:0.739 test_accuracy:0.693 test_f1_score:0.481

>> Cross Validation 2 Starts!



>> epoch:[1/3] cost:0.716 test_accuracy:0.702 test_f1_score:0.507


...


>> epoch:[2/3] cost:0.657 test_accuracy:0.775 test_f1_score:0.584



>> epoch:[3/3] cost:0.657 test_accuracy:0.801 test_f1_score:0.589

>> Cross Validation 5 Starts!



>> epoch:[1/3] cost:0.657 test_accuracy:0.812 test_f1_score:0.622



>> epoch:[2/3] cost:0.648 test_accuracy:0.842 test_f1_score:0.628



>> epoch:[3/3] cost:0.644 test_accuracy:0.824 test_f1_score:0.615

